In [1]:
#%load_ext autotime

#import matplotlib.pyplot as plt
#import pickle
import geopy
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import plotly_express as px

In [2]:
df = pd.read_csv('https://data.cityofnewyork.us/api/views/kk4q-3rt2/rows.csv')
#http://web.mta.info/developers/data/nyct/subway/Stations.csv
df.head()

,URL,OBJECTID,NAME,the_geom,LINE,NOTES
0,http://web.mta.info/nyct/service/,1,Astor Pl,POINT (-73.99106999861966 40.73005400028978),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s..."
1,http://web.mta.info/nyct/service/,2,Canal St,POINT (-74.00019299927328 40.71880300107709),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s..."
2,http://web.mta.info/nyct/service/,3,50th St,POINT (-73.98384899986625 40.76172799961419),1-2,"1-all times, 2-nights"
3,http://web.mta.info/nyct/service/,4,Bergen St,POINT (-73.97499915116808 40.68086213682956),2-3-4,"4-nights, 3-all other times, 2-all times"
4,http://web.mta.info/nyct/service/,5,Pennsylvania Ave,POINT (-73.89488591154061 40.66471445143568),3-4,"4-nights, 3-all other times"


In [3]:
#df['latlon'] = df['GTFS Latitude'].astype(str) + ', ' + df['GTFS Longitude'].astype(str)
df['latlon'] = df['the_geom'].str.lstrip('POINT (').str.rstrip(')').str.split()
df.head(60)

,URL,OBJECTID,NAME,the_geom,LINE,NOTES,latlon
0,http://web.mta.info/nyct/service/,1,Astor Pl,POINT (-73.99106999861966 40.73005400028978),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s...","[-73.99106999861966, 40.73005400028978]"
1,http://web.mta.info/nyct/service/,2,Canal St,POINT (-74.00019299927328 40.71880300107709),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s...","[-74.00019299927328, 40.71880300107709]"
2,http://web.mta.info/nyct/service/,3,50th St,POINT (-73.98384899986625 40.76172799961419),1-2,"1-all times, 2-nights","[-73.98384899986625, 40.76172799961419]"
3,http://web.mta.info/nyct/service/,4,Bergen St,POINT (-73.97499915116808 40.68086213682956),2-3-4,"4-nights, 3-all other times, 2-all times","[-73.97499915116808, 40.68086213682956]"
4,http://web.mta.info/nyct/service/,5,Pennsylvania Ave,POINT (-73.89488591154061 40.66471445143568),3-4,"4-nights, 3-all other times","[-73.89488591154061, 40.66471445143568]"
5,http://web.mta.info/nyct/service/,6,238th St,POINT (-73.90087000018522 40.88466700064975),1,"1-all times, exit only northbound","[-73.90087000018522, 40.88466700064975]"
6,http://web.mta.info/nyct/service/,7,Cathedral Pkwy (110th St),POINT (-73.95806670661364 40.800581558114956),A-B-C,"A-nights, B-weekdays and evenings, C-all times...","[-73.95806670661364, 40.800581558114956]"
7,http://web.mta.info/nyct/service/,8,Kingston - Throop Aves,POINT (-73.94085899871263 40.67991899941601),A-C,"A-nights, C-all other times","[-73.94085899871263, 40.67991899941601]"
8,http://web.mta.info/nyct/service/,9,65th St,POINT (-73.8987883783301 40.74971952935675),E-M-R,"E-nights, R-all other times, M-weekdays and ev...","[-73.8987883783301, 40.74971952935675]"
9,http://web.mta.info/nyct/service/,10,36th St,POINT (-73.92901818461539 40.75196004401078),E-M-R,"E-nights, R-all other times, M-weekdays and ev...","[-73.92901818461539, 40.75196004401078]"


In [4]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")

def reverse_geocode(coord):
    location = geolocator.reverse(coord, exactly_one=True)
    address = location.raw['address']
    try:
        zipcode = address['postcode']
    except:
        zipcode = ''
    return zipcode

print(reverse_geocode("47.470706, -99.704723"))
#reverse_geocode(df.latlon[0][1] + ', ' + df.latlon[0][0])

58418


In [5]:
df['zipcode'] = df.apply(lambda row: reverse_geocode(row['latlon'][1] + ', ' + row['latlon'][0]), axis = 1)

In [6]:
df.columns.str.replace(' ', '')
df.drop(['URL'], axis=1, inplace=True)
df.sort_values('NAME').head(50)

,OBJECTID,NAME,the_geom,LINE,NOTES,latlon,zipcode
158,159,103rd St,POINT (-73.96837899960818 40.799446000334825),1,1-all times,"[-73.96837899960818, 40.799446000334825]",10025
160,161,103rd St,POINT (-73.96137008267617 40.796060739904526),A-B-C,"A-nights, B-weekdays and evenings, C-all times...","[-73.96137008267617, 40.796060739904526]",10025-4403
457,458,103rd St,POINT (-73.94747800152219 40.79060000008452),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s...","[-73.94747800152219, 40.79060000008452]",10029
188,189,103rd St - Corona Plaza,POINT (-73.86269999830412 40.749865000555545),7,7-all times,"[-73.86269999830412, 40.749865000555545]",11368
97,98,104th St,POINT (-73.83768300060997 40.681711001091195),A-S,S Euclid Av to Ozone Park-Lefferts Blvd-nights...,"[-73.83768300060997, 40.681711001091195]",11419
13,14,104th-102nd Sts,POINT (-73.84443500029684 40.69516599823373),J-Z,"Z-rush hours AM westbound, PM eastbound, J-all...","[-73.84443500029684, 40.69516599823373]",11418
449,450,110th St,POINT (-73.94424999687163 40.795020000113105),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s...","[-73.94424999687163, 40.795020000113105]",10029
189,190,111th St,POINT (-73.85533399834884 40.75172999941711),7,7-all times,"[-73.85533399834884, 40.75172999941711]",11368
82,83,111th St,POINT (-73.83216299845388 40.68433100001238),A-S,S Euclid Av to Ozone Park-Lefferts Blvd-nights...,"[-73.83216299845388, 40.68433100001238]",11419
208,209,111th St,POINT (-73.83679338454697 40.697114810696476),J,J-all times,"[-73.83679338454697, 40.697114810696476]",11418


In [7]:
df['LINENAME'] = df.apply(lambda row: row['LINE'].rstrip('Express').replace('-',''), axis = 1)

#df.sort_values('NAME').head(20)

In [8]:
full_df = pd.read_pickle('df.p')
full_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/15/2020,00:00:00,REGULAR,7444885,2531033
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/15/2020,04:00:00,REGULAR,7444886,2531034
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/15/2020,08:00:00,REGULAR,7444899,2531046
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/15/2020,12:00:00,REGULAR,7444933,2531082
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/15/2020,16:00:00,REGULAR,7445032,2531110


In [9]:
df['STATION'] = df.apply(lambda row: row['NAME'].upper(), axis = 1)
df.sort_values('STATION').head()

,OBJECTID,NAME,the_geom,LINE,NOTES,latlon,zipcode,LINENAME,STATION
158,159,103rd St,POINT (-73.96837899960818 40.799446000334825),1,1-all times,"[-73.96837899960818, 40.799446000334825]",10025,1,103RD ST
160,161,103rd St,POINT (-73.96137008267617 40.796060739904526),A-B-C,"A-nights, B-weekdays and evenings, C-all times...","[-73.96137008267617, 40.796060739904526]",10025-4403,ABC,103RD ST
457,458,103rd St,POINT (-73.94747800152219 40.79060000008452),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s...","[-73.94747800152219, 40.79060000008452]",10029,466,103RD ST
188,189,103rd St - Corona Plaza,POINT (-73.86269999830412 40.749865000555545),7,7-all times,"[-73.86269999830412, 40.749865000555545]",11368,7,103RD ST - CORONA PLAZA
97,98,104th St,POINT (-73.83768300060997 40.681711001091195),A-S,S Euclid Av to Ozone Park-Lefferts Blvd-nights...,"[-73.83768300060997, 40.681711001091195]",11419,AS,104TH ST


In [10]:
full_df.sort_values('STATION').groupby(['STATION','LINENAME']).head(1).head(20)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
32931,H007A,R248,02-01-01,1 AV,L,BMT,09/09/2020,12:00:00,REGULAR,1,14
169527,R252,R180,00-00-01,103 ST,6,IRT,08/16/2020,05:00:00,REGULAR,5009645,2134734
49405,N037,R314,00-00-02,103 ST,BC,IND,09/04/2020,17:00:00,REGULAR,5219001,8830915
146699,R170,R191,00-00-00,103 ST,1,IRT,09/02/2020,17:00:00,REGULAR,2798316,1136801
193585,R529,R208,00-05-00,103 ST-CORONA,7,IRT,08/20/2020,16:00:00,REGULAR,13539996,5849977
69919,N137,R354,00-00-00,104 ST,A,IND,08/25/2020,04:00:00,REGULAR,414710,262245
42674,J034,R007,00-00-02,104 ST,JZ,BMT,09/07/2020,04:00:00,REGULAR,71917,52647
176742,R254,R181,01-00-02,110 ST,6,IRT,08/26/2020,12:00:00,REGULAR,14560433,4831408
42881,J035,R008,00-00-01,111 ST,J,BMT,09/01/2020,16:00:00,REGULAR,3342901,3074944
70421,N139,R355,00-00-02,111 ST,A,IND,09/11/2020,17:00:00,REGULAR,670808,145727


In [14]:
station_name_list = list(df.NAME)
station_name_list_cleaned = [entry.upper().replace('RD-','-').replace('RD ',' ').replace('ND-','-').replace('ND ',' ').replace('TH-','-').replace('TH ',' ').replace('1ST-','1-').replace('1ST ','1 ') for entry in station_name_list]
df['STATION'] = pd.Series(station_name_list_cleaned)

In [15]:
df.sort_values('STATION').head(10)

,OBJECTID,NAME,the_geom,LINE,NOTES,latlon,zipcode,LINENAME,STATION
145,146,1st Ave,POINT (-73.98168087489128 40.73097497580066),L,L-all times,"[-73.98168087489128, 40.73097497580066]",10009,L,1 AVE
158,159,103rd St,POINT (-73.96837899960818 40.799446000334825),1,1-all times,"[-73.96837899960818, 40.799446000334825]",10025,1,103 ST
160,161,103rd St,POINT (-73.96137008267617 40.796060739904526),A-B-C,"A-nights, B-weekdays and evenings, C-all times...","[-73.96137008267617, 40.796060739904526]",10025-4403,ABC,103 ST
457,458,103rd St,POINT (-73.94747800152219 40.79060000008452),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s...","[-73.94747800152219, 40.79060000008452]",10029,466,103 ST
188,189,103rd St - Corona Plaza,POINT (-73.86269999830412 40.749865000555545),7,7-all times,"[-73.86269999830412, 40.749865000555545]",11368,7,103 ST - CORONA PLAZA
97,98,104th St,POINT (-73.83768300060997 40.681711001091195),A-S,S Euclid Av to Ozone Park-Lefferts Blvd-nights...,"[-73.83768300060997, 40.681711001091195]",11419,AS,104 ST
13,14,104th-102nd Sts,POINT (-73.84443500029684 40.69516599823373),J-Z,"Z-rush hours AM westbound, PM eastbound, J-all...","[-73.84443500029684, 40.69516599823373]",11418,JZ,104-102 STS
449,450,110th St,POINT (-73.94424999687163 40.795020000113105),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s...","[-73.94424999687163, 40.795020000113105]",10029,466,110 ST
208,209,111th St,POINT (-73.83679338454697 40.697114810696476),J,J-all times,"[-73.83679338454697, 40.697114810696476]",11418,J,111 ST
82,83,111th St,POINT (-73.83216299845388 40.68433100001238),A-S,S Euclid Av to Ozone Park-Lefferts Blvd-nights...,"[-73.83216299845388, 40.68433100001238]",11419,AS,111 ST


In [18]:
zipcode_list = list(df['zipcode'])
zipcode_list_cleaned = [entry[:5] for entry in zipcode_list]
zipcode_list_cleaned

['10003',
 '10013',
 '10107',
 '11213',
 '11207',
 '10463',
 '10026',
 '11207',
 '11377',
 '11101',
 '10002',
 '11207',
 '11208',
 '11418',
 '11227',
 '11693',
 '11693',
 '10459',
 '10459',
 '10457',
 '10453',
 '10452',
 '11135',
 '11368',
 '11135',
 '10461',
 '10454',
 '10461',
 '10000',
 '10462',
 '10017',
 '10037',
 '10037',
 '10037',
 '11233',
 '11217',
 '11373',
 '11233',
 '10462',
 '10467',
 '10470',
 '11213',
 '10459',
 '10462',
 '11225',
 '11207',
 '10451',
 '10452',
 '10037',
 '10458',
 '10472',
 '10459',
 '10472',
 '11377',
 '11691',
 '10023',
 '10025',
 '10032',
 '10468',
 '10019',
 '11225',
 '11220',
 '11418',
 '11415',
 '11435',
 '11101',
 '11375',
 '11373',
 '11223',
 '11238',
 '11211',
 '11211',
 '11207',
 '11207',
 '11221',
 '11207',
 '11220',
 '11101',
 '11106',
 '10019',
 '10017',
 '11233',
 '11419',
 '10011',
 '10037',
 '10024',
 '10470',
 '10034',
 '10459',
 '10014',
 '10461',
 '10010',
 '11101',
 '',
 '11101',
 '10011',
 '10014',
 '11419',
 '11693',
 '11235',
 '111

In [19]:
df['zipcode'] = pd.Series(zipcode_list_cleaned)

In [21]:
postal_codes = df[['STATION','LINENAME', 'zipcode']]
postal_codes = postal_codes.sort_values('STATION').reset_index(drop=True)
postal_codes

,STATION,LINENAME,zipcode
0,1 AVE,L,10009
1,103 ST,1,10025
2,103 ST,ABC,10025
3,103 ST,466,10029
4,103 ST - CORONA PLAZA,7,11368
...,...,...,...
468,WOODLAWN,4,10468
469,WOODSIDE - 61 ST,77,11377
470,WORLD TRADE CENTER,,10007
471,YORK ST,F,11201


In [22]:
postal_codes_list = []
for i in range(10):
    postal_codes_list.append([postal_codes['STATION'][i], postal_codes['LINENAME'][i], postal_codes['zipcode'][i]])
print(postal_codes_list)

[['1 AVE', 'L', '10009'], ['103 ST', '1', '10025'], ['103 ST', 'ABC', '10025'], ['103 ST', '466 ', '10029'], ['103 ST - CORONA PLAZA', '7', '11368'], ['104 ST', 'AS', '11419'], ['104-102 STS', 'JZ', '11418'], ['110 ST', '466 ', '10029'], ['111 ST', 'J', '11418'], ['111 ST', 'AS', '11419']]


In [23]:
full_df.shape

(846229, 11)

In [24]:
postal_codes.shape

(473, 3)

In [25]:
newlist = ['']*10
newlist

['', '', '', '', '', '', '', '', '', '']